In [ ]:
from models.conv_vae import ConvVAE
from readers.coughvid_reader import CoughVID_Dataset
from readers.featurizer import Wave2Mel
from tools.utils import load_ckpt

In [ ]:
# 测试集读取和测试

In [ ]:
tep, tel = CoughVID_Lists(filename="./datasets/waveinfo_annotation.csv", istrain=False, isdemo=False)
print(len(tep), len(tel))

In [ ]:
tic = time.time()
test_dataset = CoughVID_Dataset(path_list=tep, label_list=tel)
toc = time.time()
print("Test Dataset Creat Completely, cost time:", toc-tic)